In [1]:
import pandas as pd
import openpyxl
import os
import re

# 读取Excel文件
df = pd.read_excel('../2023JCR.xlsx')

data_dict = {}
journal_fenqu = {}
journal_fenlei = {}
journal_5yearif = {}

#mylist = ['2024','2023','2022','2021','2020','2019']
#mylist = ['1wmore']
#mylist = ['test']
mylist = ['2018','2017','2016','2015','2014','2013','2012','2011','2010']

def get_text_before_paren(text):
    # 正则表达式匹配第一个括号或冒号前的字符
    match = re.search(r'(.*?)(\(|\:|\-)', text)
    if match:
        return match.group(1).strip()
    else:
        return text.strip()

def list_excel_files(directory):
    """
    遍历指定目录下所有以.xlsx结尾的文件，并返回文件路径列表。
    :return: 文件路径列表
    """
    return [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(directory)) for f in fn if f.endswith('.xlsx')]
        
for index, row in df.iterrows():
    value = row[df.columns[4]]   ###IF value
    key = row[df.columns[0]].title()    
    key = get_text_before_paren(key)
    key = re.sub(r'^The ', "", key)
    key = re.sub(r'/', " ", key)
    key = re.sub(r'\. ', " ", key)
    key = re.sub(r'\, ', " ", key)
    data_dict[key] = value
    journal_fenqu[key] = str(row[df.columns[7]])
    journal_fenlei[key] = str(row[df.columns[6]])
    journal_5yearif[key] = str(row[df.columns[5]])  
    
    #short journal name;
    key2 = row[df.columns[1]].title()
    key2 = get_text_before_paren(key2)
    key2 = re.sub(r'^The ', "", key2)
    key2 = re.sub(r'/', " ", key2)
    key2 = re.sub(r'\. ', " ", key2)
    key2 = re.sub(r'\, ', " ", key2)
    data_dict[key2] = value
    journal_fenqu[key2] = str(row[df.columns[7]])
    journal_fenlei[key2] = str(row[df.columns[6]])
    journal_5yearif[key2] = str(row[df.columns[5]])  

for i in mylist:
    directory = './paper_donload/' + i + '/'
    excel_files = list_excel_files(directory)
    for path in excel_files:
        wb = openpyxl.load_workbook(path)
        ws = wb["Sheet"]

        for cur_row in range(2, ws.max_row+1):
            j_name = ws.cell(row=cur_row, column=6).value
            j_short = ws.cell(row=cur_row, column=5).value
            try:
                j_name = j_name.title()
                j_name = get_text_before_paren(j_name)
                j_name = re.sub(r'^The ', "", j_name)
                j_name = re.sub(r'/', " ", j_name)
                #j_name = re.sub(r'\. ', " ", j_name)
                j_name = re.sub(r'\, ', " ", j_name)
                if j_name in data_dict:
                    ws.cell(row=cur_row, column=7).value = data_dict[j_name]
                    ws.cell(row=cur_row, column=13).value = journal_5yearif[j_name]
                    ws.cell(row=cur_row, column=14).value = journal_fenqu[j_name]
                    ws.cell(row=cur_row, column=15).value = journal_fenlei[j_name]
                elif j_short in data_dict:
                    ws.cell(row=cur_row, column=7).value = data_dict[j_short]
                    ws.cell(row=cur_row, column=13).value = journal_5yearif[j_short]
                    ws.cell(row=cur_row, column=14).value = journal_fenqu[j_short]
                    ws.cell(row=cur_row, column=15).value = journal_fenlei[j_short]
                else:
                    ws.cell(row=cur_row, column=7).value = 'NA'
                    ws.cell(row=cur_row, column=13).value = 'NA'
                    ws.cell(row=cur_row, column=14).value = 'NA'
                    ws.cell(row=cur_row, column=15).value = 'NA'
            except:     
                ws.cell(row=cur_row, column=7).value = 'NA'
                ws.cell(row=cur_row, column=13).value = 'NA'
                ws.cell(row=cur_row, column=14).value = 'NA'
                ws.cell(row=cur_row, column=15).value = 'NA'

        ws['M1'] = '5年IF'
        ws['N1'] = 'JCR分区'
        ws['O1'] = '文章类别'

        wb.save(path)
